## 프로젝트 (1) moviepy로 비디오 처리하기

In [ ]:
from moviepy.editor import VideoClip, VideoFileClip
from moviepy.editor import ipython_display
import cv2
import numpy as np
import os

### 1. moviepy를 이용해서 주피터 노트북 상에서 비디오를 읽고 쓰는 프로그램을 작성

In [ ]:
# 읽기
video_path = os.getenv('HOME')+'/aiffel/video_sticker_app/images/video2.mp4'
clip = VideoFileClip(video_path)
clip = clip.resize(width=640)
clip.ipython_display(fps=30, loop=True, autoplay=True, rd_kwargs=dict(logger=None))

# 쓰기
result_video_path = os.getenv('HOME')+'/aiffel/video_sticker_app/images/mvpyresult.mp4'
clip.write_videofile(result_video_path)

### 2. moviepy 로 읽은 동영상을 numpy 형태로 변환하고 영상 밝기를 50% 어둡게 만든 후에 저장

In [ ]:
# 읽기
video_path = os.getenv('HOME')+'/aiffel/video_sticker_app/images/video2.mp4'
clip = VideoFileClip(video_path)
clip = clip.resize(width=640)
clip.ipython_display(fps=30, loop=True, autoplay=True, rd_kwargs=dict(logger=None))

# clip 에서 numpy 로 데이터 추출
vlen = int(clip.duration*clip.fps)
video_container = np.zeros((vlen, clip.size[1], clip.size[0], 3), dtype=np.uint8)
for i in range(vlen):
    img = clip.get_frame(i/clip.fps)
    video_container[i] = (img * 0.5).astype(np.uint8)

# 새 clip 만들기
dur = vlen / clip.fps
outclip = VideoClip(lambda t: video_container[int(round(t*clip.fps))], duration=dur)

# 쓰기
result_video_path2 = os.getenv('HOME')+'/aiffel/video_sticker_app/images/mvpyresult2.mp4'
outclip.write_videofile(result_video_path2, fps=30)

### 3. 영상을 읽고 쓰는 시간,  OpenCV를 사용할 때와 차이를 측정

In [ ]:
# CASE 1 : moviepy 사용
start = cv2.getTickCount()
clip = VideoFileClip(video_path)
clip = clip.resize(width=640)

vlen = int(clip.duration*clip.fps)
video_container = np.zeros((vlen, clip.size[1], clip.size[0], 3), dtype=np.uint8)

for i in range(vlen):
    img = clip.get_frame(i/clip.fps)
    video_container[i] = (img * 0.5).astype(np.uint8)

dur = vlen / clip.fps
outclip = VideoClip(lambda t: video_container[int(round(t*clip.fps))], duration=dur)

mvpy_video_path = os.getenv('HOME')+'/aiffel/video_sticker_app/images/mvpyresult.mp4'
outclip.write_videofile(mvpy_video_path, fps=30)

time = (cv2.getTickCount() - start) / cv2.getTickFrequency()
print (f'[INFO] moviepy time : {time:.2f}ms')

In [ ]:
# CASE 2 : OpenCV 사용
start = cv2.getTickCount()
vc = cv2.VideoCapture(video_path)

cv_video_path = os.getenv('HOME')+'/aiffel/video_sticker_app/images/cvresult.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
vw = cv2.VideoWriter(cv_video_path, fourcc, 30, (640,360))

vlen = int(vc.get(cv2.CAP_PROP_FRAME_COUNT))

for i in range(vlen):
    ret, img = vc.read()
    if ret == False: break
    
    img_result = cv2.resize(img, (640, 360)) * 0.5
    vw.write(img_result.astype(np.uint8))
    
time = (cv2.getTickCount() - start) / cv2.getTickFrequency()
print (f'[INFO] cv time : {time:.2f}ms')

### 4. moviepy 를 이용할 때의 장단점을 분석

1. 장점
    - 여러 비디오를 처리하거나 복잡한 방식으로 합칠 때 유용하다.
    - video effect를 효율적으로 추가할 수 있다.
2. 단점
    - frame 단위로 분석할 땐 OpenCV에 비해 느리고 비효율적이다.

## 프로젝트 (2) 어디까지 만들고 싶은지 정의하기

### 1. 실시간 카메라 스티커앱 만들기 (웹캠 입력을 사용)

webcam_sticker.py

### 2. 스티커앱을 실행하고 카메라와의 거리를 조정해, 얼굴을 찾지 못하는 거리를 기록

정확한 측정 장비로 측정하진 않았지만, 대략 15cm보다 좀 더 가까워지면 스티커가 사라졌다. 최대 거리는 생각 외로 길었고, 200cm 정도 되는 것 같다.

### 3. 고개를 상하좌우로 움직여 yaw, pitch, roll 각도의 개념을 직접 실험해 보고 각각 몇 도까지 정상적으로 스티커앱이 동작하는지 기록

(참고)
- yaw : y축 기준 회전 → 높이 축
- picth : x축 기준 회전 → 좌우 축
- roll : z축 기준 회전 → 거리 축

- yaw : -45 ~ 45도
- pitch : -20 ~ 30도
- roll : -45 ~ 45도

직접 측정해봤을 때, 범위는 위와 같았으나, yaw, pitch, roll 모두 양 극단에 미치지 못하는 느낌이 있었다. 예를 들어 극단 값이 45도면 40도 정도인 느낌이었다.

### 4. 만들고 싶은 스티커앱의 스펙(허용 거리, 허용 인원 수, 허용 각도, 안정성)

- 거리 : 30cm ~ 1m → 너무 가까우면 스티커의 의미가 없음, 셀카봉을 들었을 때의 유효거리
- 인원 수 : 2명. 일반적으로 스티커앱은 1~2명이 주된 사용 인원이라고 생각하며, 그 이상으로 넘어가면 거리의 최대값도 조정이 필요하다.
- 허용 각도 : pitch : -20 ~ 30도, yaw : -45 ~ 45도, roll : -45 ~ 45도 → 위의 3. 에서 측정해봤을 때, 해당 범위들을 넘어서면 얼굴이 일부분만 나오게 되고, 일반적으로 스티커 사진을 사용할 땐 눈, 코, 입이 온전히 나오는 상황이 많음을 고려.
- 안정성 : 위 조건을 만족하면서 FPPI (false positive per image) 기준 < 0.003, MR (miss rate) < 1 300장당 1번 에러 = 10초=30*10에 1번 에러

## 프로젝트 (3) 스티커 Out Bound 예외처리 하기

### 1. 지금까지 만든 스티커앱을 이용해서 예외 상황 찾기. 특히 서서히 영상에서 좌우 경계 밖으로 나가며 코드의 행동을 확인.

왼쪽 경계 밖으로 나가면 다음과 같은 에러가 발생한다.
```
Traceback (most recent call last):
  File "webcam_sticker.py", line 31, in <module>
    main()
  File "webcam_sticker.py", line 23, in main
    img_result = img2sticker(img, img_sticker.copy(), detector_hog, landmark_predictor)
  File "/home/aiffel/aiffel/video_sticker_app/newaddsticker.py", line 34, in img2sticker
    cv2.addWeighted(sticker_area, 1.0, img_sticker, 0.7, 0)
cv2.error: OpenCV(4.0.1) C:\ci\opencv-suite_1573470242804\work\modules\core\src\arithm.cpp:663: error: (-209:Sizes of input arguments do not match) The operation is
neither 'array op array' (where arrays have the same size and the same number of channels), nor 'array op scalar', nor 'scalar op array' in function 'cv::arithm_op'
```

### 2. 문제가 어디에서 발생하는지 코드에서 확인

```
sticker_area = img_bgr[refined_y:refined_y+img_sticker.shape[0], refined_x:refined_x+img_sticker.shape[1]]
```

### 3. Out bound 오류(경계 밖으로 대상이 나가서 생기는 오류)를 해결.

```
import dlib
import cv2

def img2sticker(img_orig, img_sticker, detector_hog, landmark_predictor):
    # preprocess
    img_rgb = cv2.cvtColor(img_orig, cv2.COLOR_BGR2RGB)
    
    # detector
    dlib_rects = detector_hog(img_rgb, 0)
    if len(dlib_rects) < 1:
        return img_orig
    
    # landmark
    list_landmarks = []
    for dlib_rect in dlib_rects:
        points = landmark_predictor(img_rgb, dlib_rect)
        list_points = list(map(lambda p: (p.x, p.y), points.parts()))
        list_landmarks.append(list_points)
    
    # head coord
    for dlib_rect, landmark in zip(dlib_rects, list_landmarks):
        x = landmark[30][0] # nose
        y = landmark[30][1] - dlib_rect.width()//2
        w = dlib_rect.width()
        h = dlib_rect.width()
        break
    
    # sticker
    img_sticker = cv2.resize(img_sticker, (w,h), interpolation=cv2.INTER_NEAREST)
    
    refined_x = x - w // 2
    refined_y = y - h
    
    if refined_y < 0:
        img_sticker = img_sticker[-refined_y:]
        refined_y = 0

    if refined_x < 0:
        img_sticker = img_sticker[:, -refined_x:]
        refined_x = 0
    elif refined_x + img_sticker.shape[1] >= img_orig.shape[1]:
        img_sticker = img_sticker[:, :-(img_sticker.shape[1]+refined_x-img_orig.shape[1])]
        
    img_bgr = img_orig.copy()
    sticker_area = img_bgr[refined_y:refined_y+img_sticker.shape[0], refined_x:refined_x+img_sticker.shape[1]]

    img_bgr[refined_y:refined_y+img_sticker.shape[0], refined_x:refined_x+img_sticker.shape[1]] = \
        cv2.addWeighted(sticker_area, 1.0, img_sticker, 0.7, 0)

    return img_bgr

```

### 4. 다른 예외는 어떤 것들이 있는지 정의

고개를 좌우로 돌렸을 때 얼굴을 기준으로 스티커를 붙이므로 왕관 스티커가 이마쪽에 위치하는 것

## 프로젝트 (4) 스티커앱 분석 - 거리, 인원 수, 각도, 시계열 안정성


### 1. 멀어지는 경우에 왜 스티커앱이 동작하지 않는지 분석

dlib detection 문제. 멀어지면 detector_hog 단계에서 bbox 가 출력되지 않음.

```
# preprocess
img_rgb = cv2.cvtColor(img_orig, cv2.COLOR_BGR2RGB)
# detector
img_rgb_vga = cv2.resize(img_rgb, (640, 360))
dlib_rects = detector_hog(img_rgb_vga, 0)
if len(dlib_rects) < 1:
    return img_orig
```

### 2. 이미지 피라미드를 조절하여 성능을 향상시키는 간단한 방법을 활용하여 img2sticker 메소드를 간단히 고쳐 보기


```
def img2sticker(img_orig, img_sticker, detector_hog, landmark_predictor):
    # preprocess
    img_rgb = cv2.cvtColor(img_orig, cv2.COLOR_BGR2RGB)

    # detector
    img_rgb_vga = cv2.resize(img_rgb, (640, 360))
    dlib_rects = detector_hog(img_rgb_vga, 1) # <- 이미지 피라미드 수 변경
    if len(dlib_rects) < 1:
        return img_orig

    # landmark
    list_landmarks = []
    for dlib_rect in dlib_rects:
        points = landmark_predictor(img_rgb_vga, dlib_rect)
        list_points = list(map(lambda p: (p.x, p.y), points.parts()))
        list_landmarks.append(list_points)
```

### 3. 위에서 새롭게 시도한 방법의 문제점

속도가 현저히 느려짐. 기존 30ms/frame 에서 120ms/frame 으로 약 4배 느려짐 → 실시간 구동이 불가능.

### 4. 실행시간을 만족할 수 있는 방법

hog 디텍터를 딥러닝 기반 디텍터로 변경할 수 있다. hog 학습 단계에서 다양한 각도에 대한 hog 특징을 모두 추출해서 일반화 하기 어렵기 때문에 딥러닝 기반 검출기의 성능이 훨씬 좋다.

딥러닝 기반 detection 방법을 조사한다.   

opencv 는 intel cpu 을 사용할 때 dnn 모듈이 가속화를 지원하고 있다. 따라서 mobilenet 과 같은 작은 backbone 모델을 사용하고 ssd 를 사용한다면 충분히 만족할 만한 시간과 성능을 얻을 수 있다.

### 5. 인원 수, 각도 등 각 문제에 대해서 문제점과 대안 제시

**인원 수**

- 문제점: \# head coord 부분의 반복문에서 첫 번째 얼굴의 좌표만 담고 break해서 탈출해, 얼굴을 여러 개 인식하더라도 하나만 활용.
- 대안책: break하지 않고 모든 dlib_rect의 x, y, w, h 값을 리스트 등으로 담아 아래의 코드에서 반복문으로 활용.

**각도**

- 문제점: 정면을 바라보지 않고 각도를 틀어도 스티커 이미지는 동일한 형태로 적용되는 것이 문제다.
- 대안책: 랜드마크 중 단순히 코(30번) 하나만 활용할 것이 아니라, 여러 기준점들을 활용해 각도가 틀어질 때 기준점 위치들의 변화에 따라 스티커 이미지도 선형 변환 시킨다.

## 프로젝트 (5) 칼만 필터 적용하기

### 1. 카메라 앞에서 가만히 있을 때 스티커의 움직임을 관찰

가만히 있어도 스티커의 크기가 일정하게 유지되지 않고, 떨리는 것처럼 보이는 현상이 발생

### 2. 이론 강의에서 배운 칼만 필터를 적용해서 스티커 움직임을 안정화

```
import numpy as np
import cv2
import dlib

from kfaddsticker import img2sticker_kf

detector_hog = dlib.get_frontal_face_detector()
landmark_predictor = dlib.shape_predictor('./models/shape_predictor_68_face_landmarks.dat')

def main():
    cv2.namedWindow('show', 0)
    cv2.resizeWindow('show', 640, 360)

    vc = cv2.VideoCapture(0)
    img_sticker = cv2.imread('./images/king.png')

    vlen = int(vc.get(cv2.CAP_PROP_FRAME_COUNT))
    print (vlen) # 웹캠은 video length 가 0 입니다.

    # 정해진 길이가 없기 때문에 while 을 주로 사용합니다.
    # for i in range(vlen):
    while True:
        ret, img = vc.read()
        if ret == False:
            break
        start = cv2.getTickCount()
        img = cv2.flip(img, 1)  # 보통 웹캠은 좌우 반전

        # 스티커 메소드를 사용
        img_result = img2sticker_kf(img, img_sticker.copy(), detector_hog, landmark_predictor)   

        time = (cv2.getTickCount() - start) / cv2.getTickFrequency() * 1000
        print ('[INFO] time: %.2fms'%time)
        
        cv2.imshow('show', img_result)
        key = cv2.waitKey(1)
        if key == 27:
            break


if __name__ == '__main__':
    main()
```

### 회고

칼만필터가 좀 아쉽다.   
유효한 효과를 거뒀다고 보기 어려운 결과였다.